# 06 - Multi-Agent Systems

**Orchestrate teams of AI agents!** Learn how multiple specialized agents can collaborate.

## Learning Objectives

By the end of this notebook, you will:
- Understand agent roles and specialization
- Implement sequential and parallel agent execution
- Build orchestration patterns
- Manage shared state between agents

## Table of Contents

1. [Why Multi-Agent?](#why)
2. [Agent Specialization](#specialization)
3. [Communication Patterns](#patterns)
4. [Building an Orchestrator](#orchestrator)
5. [Exercises](#exercises)
6. [Checkpoint](#checkpoint)

In [ ]:
# GUIDED: Setup
import os
import sys
import json
from pathlib import Path
from dataclasses import dataclass
from typing import Optional

sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / ".env")

from openai import OpenAI
client = OpenAI()

print("Setup complete!")

---
## 1. Why Multi-Agent? <a id='why'></a>

### Single Agent Limitations

- **Context overload**: Too many responsibilities
- **Jack of all trades**: Can't be expert at everything
- **Error propagation**: One mistake affects everything

### Multi-Agent Benefits

- **Specialization**: Each agent excels at specific tasks
- **Parallelization**: Independent tasks run simultaneously
- **Modularity**: Easy to update individual agents
- **Robustness**: Failure isolation

In [ ]:
# GUIDED: Example scenario - Content Creation Pipeline

scenario = """
CONTENT CREATION PIPELINE
=========================

Task: Write a blog post about AI agents

Single Agent Approach:
  - One agent does everything: research, outline, write, edit
  - Prone to inconsistency
  - No checks and balances

Multi-Agent Approach:
  ┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
  │  Researcher │ --> │   Outliner  │ --> │   Writer    │ --> │   Editor    │
  │   Agent     │     │    Agent    │     │   Agent     │     │   Agent     │
  └─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
       │                    │                   │                   │
       │                    │                   │                   │
       ▼                    ▼                   ▼                   ▼
    Research            Structured          Draft             Polished
    Notes                Outline            Article           Article
"""

print(scenario)

---
## 2. Agent Specialization <a id='specialization'></a>

Each agent has a specific role with tailored prompts and capabilities.

In [ ]:
# GUIDED: Define specialized agents

@dataclass
class SimpleAgent:
    """A simple specialized agent."""
    name: str
    role: str
    system_prompt: str
    
    def run(self, task: str, context: str = "") -> str:
        """Execute the agent on a task."""
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"{context}\n\nTask: {task}" if context else f"Task: {task}"}
        ]
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.7
        )
        
        return response.choices[0].message.content

# Create specialized agents
researcher = SimpleAgent(
    name="Researcher",
    role="research",
    system_prompt="""You are a research specialist. Your job is to:
- Gather key facts and information
- Identify important points
- Provide source-worthy insights

Output a concise research summary with bullet points."""
)

writer = SimpleAgent(
    name="Writer",
    role="writing",
    system_prompt="""You are a content writer. Your job is to:
- Write clear, engaging content
- Use the research provided as your source
- Structure content with headers and paragraphs

Write in a professional but accessible tone."""
)

editor = SimpleAgent(
    name="Editor",
    role="editing",
    system_prompt="""You are an editor. Your job is to:
- Review content for clarity and flow
- Fix grammar and style issues
- Suggest improvements
- Provide the polished final version

Return the improved content with any necessary changes."""
)

print("Agents created: Researcher, Writer, Editor")

In [ ]:
# GUIDED: Test individual agents

print("Testing Researcher Agent:")
print("=" * 40)
research = researcher.run("Research the benefits of AI agents")
print(research)

---
## 3. Communication Patterns <a id='patterns'></a>

### Sequential (Pipeline)
```
Agent A → Agent B → Agent C → Result
```

### Parallel (Fan-out/Fan-in)
```
        ┌→ Agent B ─┐
Agent A ┼→ Agent C ─┼→ Agent E
        └→ Agent D ─┘
```

### Hierarchical (Manager/Workers)
```
        Manager Agent
       /      |      \
  Worker   Worker   Worker
```

In [ ]:
# GUIDED: Sequential pipeline

def run_pipeline(task: str, agents: list[SimpleAgent], verbose: bool = True) -> str:
    """
    Run agents in sequence, passing output to next agent.
    """
    context = ""
    
    for agent in agents:
        if verbose:
            print(f"\n{'='*50}")
            print(f"Running: {agent.name}")
            print('='*50)
        
        result = agent.run(task, context)
        
        if verbose:
            print(f"\n{result[:500]}..." if len(result) > 500 else f"\n{result}")
        
        # Build context for next agent
        context = f"Previous agent ({agent.name}) output:\n{result}"
    
    return result

# Run the content pipeline
final_content = run_pipeline(
    task="Write a short article about AI agents",
    agents=[researcher, writer, editor]
)

In [ ]:
# GUIDED: Parallel execution (simulated)

def run_parallel(task: str, agents: list[SimpleAgent]) -> dict[str, str]:
    """
    Run multiple agents in parallel on the same task.
    Returns results from all agents.
    """
    # In production, use asyncio or threading
    # This is a simplified sequential simulation
    results = {}
    
    for agent in agents:
        print(f"Running {agent.name}...")
        results[agent.name] = agent.run(task)
    
    return results

# Create parallel agents
analyst1 = SimpleAgent(
    name="Technical Analyst",
    role="analysis",
    system_prompt="Analyze from a technical perspective. Focus on implementation details."
)

analyst2 = SimpleAgent(
    name="Business Analyst",
    role="analysis", 
    system_prompt="Analyze from a business perspective. Focus on ROI and value."
)

# Run parallel analysis
results = run_parallel(
    "Analyze the potential of AI agents in customer service",
    [analyst1, analyst2]
)

print("\n" + "="*50)
print("Combined Results:")
for agent_name, result in results.items():
    print(f"\n{agent_name}:")
    print(result[:300] + "...")

---
## 4. Building an Orchestrator <a id='orchestrator'></a>

An orchestrator manages agent execution and state.

In [ ]:
# GUIDED: Simple orchestrator

class SimpleOrchestrator:
    """Orchestrates multiple agents."""
    
    def __init__(self):
        self.agents: dict[str, SimpleAgent] = {}
        self.shared_state: dict = {}
        self.execution_log: list = []
    
    def register(self, agent: SimpleAgent) -> None:
        """Register an agent."""
        self.agents[agent.name] = agent
        print(f"Registered: {agent.name}")
    
    def run_agent(self, agent_name: str, task: str) -> str:
        """Run a specific agent."""
        if agent_name not in self.agents:
            raise ValueError(f"Unknown agent: {agent_name}")
        
        agent = self.agents[agent_name]
        
        # Build context from shared state
        context = ""
        if self.shared_state:
            context = "Shared context:\n" + json.dumps(self.shared_state, indent=2)
        
        result = agent.run(task, context)
        
        # Log execution
        self.execution_log.append({
            "agent": agent_name,
            "task": task,
            "result": result[:200]
        })
        
        return result
    
    def update_state(self, key: str, value: any) -> None:
        """Update shared state."""
        self.shared_state[key] = value
    
    def run_workflow(self, workflow: list[tuple[str, str]]) -> dict[str, str]:
        """
        Run a workflow of (agent_name, task) pairs.
        """
        results = {}
        
        for agent_name, task in workflow:
            print(f"\nExecuting: {agent_name}")
            result = self.run_agent(agent_name, task)
            results[agent_name] = result
            
            # Store in shared state
            self.update_state(f"{agent_name}_output", result)
        
        return results

# Create and configure orchestrator
orch = SimpleOrchestrator()
orch.register(researcher)
orch.register(writer)
orch.register(editor)

In [ ]:
# GUIDED: Run a workflow

workflow = [
    ("Researcher", "Research benefits of AI in healthcare"),
    ("Writer", "Write a short summary based on the research"),
    ("Editor", "Polish the summary")
]

results = orch.run_workflow(workflow)

print("\n" + "="*50)
print("Final Output:")
print("="*50)
print(results["Editor"])

In [ ]:
# GUIDED: Use our framework's orchestrator

from src.llm_client import LLMClient
from src.tool_registry import ToolRegistry
from src.agent_framework import ReActAgent, AgentOrchestrator, AgentTask

# Create orchestrator from our framework
orchestrator = AgentOrchestrator(verbose=True)

# Create specialized ReAct agents
llm = LLMClient(provider="openai", model="gpt-4o-mini")

research_agent = ReActAgent(
    name="research",
    llm=llm,
    tools=ToolRegistry(),
    system_prompt="You are a research agent. Gather and summarize information.",
    max_steps=3,
    verbose=False
)

writing_agent = ReActAgent(
    name="writer",
    llm=llm,
    tools=ToolRegistry(),
    system_prompt="You are a writing agent. Create clear, engaging content.",
    max_steps=3,
    verbose=False
)

orchestrator.register_agent("research", research_agent)
orchestrator.register_agent("writer", writing_agent)

print("\nRegistered agents:", orchestrator.list_agents())

---
## 5. Exercises <a id='exercises'></a>

### Exercise 1: Create a Review Agent

Add a "Reviewer" agent that critiques content.

In [ ]:
# TODO: Create a Reviewer agent and add it to the pipeline
# The reviewer should provide constructive feedback

# Your code here:


### Exercise 2: Parallel Analysis

Create 3 analysts with different perspectives and combine their insights.

In [ ]:
# TODO: Create 3 analyst agents (e.g., Technical, Business, User Experience)
# Run them in parallel and create a synthesizer agent to combine insights

# Your code here:


### Exercise 3: Debate System

Create two agents that debate a topic, taking opposing views.

In [ ]:
# TODO: Create a Pro and Con agent
# Have them debate for 2-3 rounds
# Create a Judge agent to summarize the debate

# Your code here:


---
## 6. Checkpoint <a id='checkpoint'></a>

Before moving on, verify:

- [ ] You understand why multi-agent systems are useful
- [ ] You can create specialized agents
- [ ] You implemented sequential and parallel patterns
- [ ] You built a simple orchestrator
- [ ] You completed at least 2 exercises

### Next Steps

In the next notebook, we'll explore **Autonomous Workflows** - creating agents that can plan and execute complex tasks!

---
## Summary

**Multi-Agent Patterns:**

| Pattern | Use Case |
|---------|----------|
| Sequential | Pipeline processing |
| Parallel | Independent analysis |
| Hierarchical | Manager/worker tasks |

**Key Components:**
- Specialized agents with focused prompts
- Orchestrator for coordination
- Shared state for communication
- Execution logging

**Best Practices:**
- Keep agents focused on single responsibilities
- Use clear interfaces between agents
- Log all inter-agent communication
- Handle agent failures gracefully